# Road Follower - Train Model

This is Host side Jupyter Notebook for project located under the name "road_follower" on your JetBot. We will use this notebook to train road following model. In this project we will be using Regression (continuous value output) to train JetBot where to move steering! 

We will be using PyTorch deep learning framework to train ResNet18 neural network architecture model for road follower application.

In [12]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Download and extract data

Before you start, you should upload the ``dataset_roadfollower_{date_time}.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot. 

You can use scp to download data from your JetBot as follows:

`` scp jetbot@<ip_address>:<dataset_directory_path> . ``

You should then extract this dataset by calling the command below:

In [2]:
!unzip -q lego_city_dataset_all_2019-03-16_12-52-20.zip

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance
Now we use the abstract class representing a dataset available with the ``torch.utils.data.Dataset`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.

Each file in dataset is labelled with ``steering_<>_<>_<>_<>_<>_<>.jpg`` 
Thus, we will use from 9 to 12 as steering label.

In [13]:
def get_steering(path):
    return (float(int(path[9:12])) - 50.0) / 50.0

def get_throttle(path):
    return (float(int(path[12:15])) - 50.0) / 50.0
class SteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, stdev=0.3, nbins=21):
        self.nbins = nbins
        self.stdev = stdev
        self.directory = directory
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        steering = float(get_steering(os.path.basename(image_path)))   
        throttle = float(get_throttle(os.path.basename(image_path)))
        return image, torch.tensor([steering]).float(), torch.tensor([throttle]).float()

dataset = SteeringDataset('dataset_all')

### Split dataset into train and test sets
Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

In [14]:
test_percent = 0.1
BATCH_SIZE = 64
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches

We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model. 

In [15]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

### Define Neural Network Model 

We use ResNet-18 model available on PyTorch TorchVision. 

In a process called transfer learning, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.


More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

In [16]:
model = models.resnet18(pretrained=True)

ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

In [17]:
model.fc = torch.nn.Linear(512, 1)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

We train for 50 epochs and save best model if the loss is reduced. 

In [18]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_forward.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('Epoch : %d, Test Loss: %f' % (epoch, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

Epoch : 0, Test Loss: 168.965347
Epoch : 1, Test Loss: 14.124955
Epoch : 2, Test Loss: 36.150612
Epoch : 3, Test Loss: 41.710323
Epoch : 4, Test Loss: 17.329863
Epoch : 5, Test Loss: 7.215171
Epoch : 6, Test Loss: 3.752449
Epoch : 7, Test Loss: 2.276010
Epoch : 8, Test Loss: 0.891368
Epoch : 9, Test Loss: 0.274711
Epoch : 10, Test Loss: 0.157284
Epoch : 11, Test Loss: 0.083652
Epoch : 12, Test Loss: 0.029237
Epoch : 13, Test Loss: 0.024496
Epoch : 14, Test Loss: 0.018810
Epoch : 15, Test Loss: 0.017174
Epoch : 16, Test Loss: 0.018829
Epoch : 17, Test Loss: 0.016731
Epoch : 18, Test Loss: 0.013730
Epoch : 19, Test Loss: 0.014714
Epoch : 20, Test Loss: 0.015781
Epoch : 21, Test Loss: 0.015388
Epoch : 22, Test Loss: 0.015901
Epoch : 23, Test Loss: 0.016596
Epoch : 24, Test Loss: 0.015942
Epoch : 25, Test Loss: 0.013331
Epoch : 26, Test Loss: 0.013768
Epoch : 27, Test Loss: 0.015897
Epoch : 28, Test Loss: 0.014649
Epoch : 29, Test Loss: 0.014428
Epoch : 30, Test Loss: 0.012016
Epoch : 31, 

Once the model is trained, it will generate ``best_steering_model_forward.pth`` file which you can use for inferencing.

You can use scp to upload your model file (.pth) to JetBot as follows:

`` scp best_steering_model_forward.pth jetbot@<ip_address>:<inference_path>  ``

or

 Select ``Right click`` -> ``Download`` to download the model to your workstation